<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/crawer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網路爬蟲與多執行緒練習

未穩定的版本

In [ ]:
#@title UU看書 專用(多執行緖)
#@markdown 還在修正的程式，可以直接從這一個區塊執行


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  
  soup=BeautifulSoup(requests.get(art_url).text)
  content=soup.find(name="div",id='contentbox')
  print(art_id)
  text=f"{title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="".join(context)+"\n\n"

  
  
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)
  

#@markdown 書籍目錄網址
url="https://tw.uukanshu.com/b/109951/" #@param {type:'string'}
sites=url[:url.find("/",8)]

reg=requests.get(url)
soup=BeautifulSoup(reg.text,"html.parser")
output_name=soup.find("h2").getText()
articles=soup.find(name="ul",id="chapterList").find_all("a")

links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  links.append([i.get("title"),f"{sites}{href}"])
links.sort(key=lambda x: x[1])

for url in links[:4]:
  get_html(url)


# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(get_html, links)

output_name=soup.find("h2").getText()
files_text=os.listdir()
files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
files_text.sort(key=lambda x:int(x[:-4]))
with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
  for file in files_text:
    with open(file,"r") as f2:
      f.write(f2.read())

from google.colab import files
files.download('../{}.txt'.format(output_name)) 

# 效率

透過下述的方法，合併檔案，因為輸出檔需要被反覆的開始太多次，隨著檔案大小逐漸增加。讓效能下跌

```python
for file in files:
  os.system("cat {}>> ../{}.txt".format(file,output_name))
```
若改用下述的方法， output 檔，只需要開啟一次。可以大大縮短時間。

```python
with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
  for file in files_text:
    with open(file,"r") as f2:
      f.write(f2.read())
```

In [ ]:
#@title 品書閣 專用(多執行緖)
#@markdown 還在修正的程式，可以直接從這一個區塊執行

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")

stop_word="請大家收藏" 

  
def get_html(arr):
  
  [titles,links]=arr 
  art_id=links[links.rfind("/")+1:-5]

  soup=BeautifulSoup(requests.get(links).text)
  content=soup.find(name="div",id='content').find_all("p")

  text=f"{titles}\n"

  for itm in content[1:]:
    txt=itm.getText()
    if "一秒記住" in txt:
      continue
    if "點下一章繼續閱讀" in txt:
      text+=f"{txt}\n"
      break
    if stop_word in txt:
      break    
    # text+="%s \n" %txt
    
    text+=f"{txt}\n"
  
  # print(art_id)
  next_article=BeautifulSoup(requests.get(links).text).find("a",text="下一章").get("href")
  

  if art_id in next_article:
    
    soup2=BeautifulSoup(requests.get(f"{sites}{next_article}").text)
    content2=soup2.find(name="div",id="content").find_all("p")
    print("***",next_article)
    for itm2 in content2:
      txt2=itm2.getText()
      if "一秒記住" in txt2:
        continue
      if stop_word in txt2:
        break          
      text+=f"{txt2}\n"
   
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://tw.pinsuge.com/index/85767.html" #@param {type:'string'}
sites=url[:url.find("/",8)]
req = requests.get(url)
soup=BeautifulSoup(req.text)
articles=soup.find_all("a")
links=[]
titles=[]
for itm in articles[1:]:
  if not itm.get("title"):
    continue 
  links.append([itm.get("title",),"{}{}".format(sites,itm.get("href"))])

# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(get_html, links)

files_text=os.listdir()
files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
files_text.sort(key=lambda x:int(x[:-4]))
with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
  for file in files_text:
    with open(file,"r") as f2:
      f.write(f2.read())
      
from google.colab import files
files.download('../{}.txt'.format(output_name)) 


# 參考資料

In [ ]:
#@title BeauttifulSoup 練習
#@markdown extract, decompose 的練習
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>


<p class="story">Once upon a time there were three little sisters; and their names were <script>.lkjlkjlj\n\r
我是\n</script> <i> hello world</i>
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""
from bs4 import BeautifulSoup
soup=BeautifulSoup(html_doc,"lxml")
txt=soup.find("p","story")
print(str(txt))
print("".center(100,"-"))
txt.script.decompose()

txt.a.extract()
txt.a.extract()

print(str(txt))

In [ ]:
#@title 品書閣 (舊) 單執行緖版本
#@markdown 查看目錄=>章節列表
import requests
import os,re,time
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")

url="https://tw.pinsuge.com/index/85767.html" #@param {type:'string'}
sites=url[:url.find("/",8)]
req = requests.get(url)
#@markdown 在 stop_word 之後為相關的廣告內容
stop_word="請大家收藏" #@param {type:'string'}
#@markdown 若章節分成兩個頁面的處理字元
next_word="點下一章繼續" #@param {type:'string'}
soup=BeautifulSoup(req.text)

soup
articles=soup.find_all(name="a")
output_name=soup.find("h1").getText()
links=[]

for itm in articles[1:]:
  if not itm.get("title"):
    continue 
  links.append([itm.get("title",),"{}{}".format(sites,itm.get("href"))])


def get_content(content):
  to_text=''
  for itm in content[:-1]:
    txt=itm.getText()
    if "一秒記住" in txt:
      continue
    if stop_word in txt:
      break    
    to_text+="%s \n" %txt
  return to_text

def get_soup(url):
  return BeautifulSoup(requests.get(url).text)

def cont_chapter_check(soup,text):
  return (len(soup.find_all(name="p",string=re.compile(text))) >0)

def cont_chapter_link(soup):
  return soup.find("a",text="下一章").get("href")

index=1
star=time.time()
for link in links:  
  soup=get_soup(link[1])
  to_text=''
  to_text+="%s \n" %link[0]
  print(link[0])
  to_text+=get_content(soup.find(name="div",id="content").find_all(name="p"))
  
  if cont_chapter_check(soup,next_word):
    reg2=requests.get("{}{}".format(sites,cont_chapter_link(soup)))
    to_text+=get_content(BeautifulSoup(reg2.text).find(name="div",id="content").find_all(name="p"))
  # print(to_text)

  with open("%03d.txt" % index,mode="w",encoding="utf-8") as f:
    f.write(to_text)
  index+=1

files=os.listdir()
files=[file for file in files if file.endswith(".txt")]
files.sort()
files
for file in files:
  os.system("cat {}>> ../{}.txt".format(file,output_name))

from google.colab import files
files.download('../{}.txt'.format(output_name)) 
end=time.time()
print("經過了 {end-time} 秒")

In [ ]:
#@title 多執行序參考程式範例

from bs4 import BeautifulSoup
import concurrent.futures
import requests
import time
 
 
def scrape(urls):
  
    response = requests.get(urls)
 
    soup = BeautifulSoup(response.content, "lxml")
 
    # 爬取文章標題
    titles = soup.find_all("h3", {"class": "post_title"})
 
    for title in titles:
        print(title.getText().strip())
 
    time.sleep(2)
 
 
base_url = "https://www.inside.com.tw/tag/AI"
urls = [f"{base_url}?page={page}" for page in range(1, 6)]  # 1~5頁的網址清單
print(urls) 
start_time = time.time()  # 開始時間
# scrape(urls)

 
# 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#     executor.map(scrape, urls)
 
end_time = time.time()
print(f"{end_time - start_time} 秒爬取 {len(urls)} 頁的文章")